In [ ]:
"""
AI Exposure and College Enrollment Analysis - 4-DIGIT CIP VERSION
==================================================================
Updated for:
- 4-digit CIP codes (436 programs vs 49 at 2-digit level)
- 2019-2025 enrollment data (combined from both files)
- More granular analysis (e.g., Computer Science vs Information Systems)
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# STEP 1: LOAD FELTEN AIOE DATA
# =============================================================================

def load_felten_data(filepath: str) -> pd.DataFrame:
    """Load Felten et al. (2021) AIOE scores."""
    print("="*70)
    print("LOADING FELTEN AIOE DATA")
    print("="*70)
    
    felten = pd.read_excel(filepath, sheet_name='Appendix A')
    felten['soc_clean'] = felten['SOC Code'].str.replace('-', '').str.replace('.', '')
    
    print(f"\nLoaded {len(felten)} occupations")
    print(f"AIOE range: {felten['AIOE'].min():.2f} to {felten['AIOE'].max():.2f}")
    
    return felten


# =============================================================================
# STEP 2: LOAD FOD TO 4-DIGIT CIP CROSSWALK
# =============================================================================

def load_fod_cip4_crosswalk(filepath: str) -> Dict[int, str]:
    """
    Load FOD to 4-digit CIP mapping from crosswalk file.
    
    The crosswalk has detailed 6-digit CIP codes (like 11.0701).
    We extract 4-digit CIP:
    - Family (2 digits): 11 = Computer Science
    - Group (next 2 digits): 07 = Computer Science 
    - Combined: 1107 = Computer Science (4-digit)
    
    Examples:
    - 11.0000 → 1100 (Computer Science, General)
    - 11.0701 → 1107 (Computer Science)
    - 52.0201 → 5202 (Business Administration)
    
    Returns:
    --------
    Dict mapping FOD code (int) to 4-digit CIP string (e.g., '1107')
    """
    print("\n" + "="*70)
    print("LOADING FOD TO 4-DIGIT CIP CROSSWALK")
    print("="*70)
    
    # Read from "CIP code by HHES code" sheet
    df = pd.read_excel(filepath, sheet_name='CIP code by HHES code', skiprows=1)
    
    # Extract FOD and CIP columns
    crosswalk = df[['HHES Code', 'CIP \nCode']].copy()
    crosswalk.columns = ['FOD', 'CIP']
    crosswalk = crosswalk.dropna(subset=['FOD', 'CIP'])
    
    # Convert FOD to integer
    crosswalk['FOD'] = crosswalk['FOD'].astype(int)
    
    # Extract 4-digit CIP from 6-digit CIP code
    # CIP format: XX.XXXX where first 2 are family, next 2 are group
    # E.g., 11.0701 → 1107
    def extract_cip4(cip_6digit):
        try:
            cip_float = float(cip_6digit)
            # Get integer part (family, 2 digits)
            family = int(cip_float)  # e.g., 11
            # Get first 2 decimal digits (group)
            decimal_part = cip_float - family  # e.g., 0.0701
            # Extract first 2 decimal digits
            group = int(round(decimal_part * 10000)) // 100  # e.g., 07
            # Combine to 4-digit code
            cip4 = f"{family:02d}{group:02d}"  # e.g., "1107"
            return cip4
        except:
            return None
    
    crosswalk['CIP4'] = crosswalk['CIP'].apply(extract_cip4)
    crosswalk = crosswalk.dropna(subset=['CIP4'])
    
    # For each FOD, use the modal (most common) 4-digit CIP
    # (some FOD codes map to multiple detailed CIPs within same 4-digit group)
    fod_to_cip4 = crosswalk.groupby('FOD')['CIP4'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0]).to_dict()
    
    print(f"\nMapped {len(fod_to_cip4)} FOD codes to 4-digit CIP codes")
    
    # Show sample mappings
    print("\nSample mappings:")
    for fod in sorted(fod_to_cip4.keys())[:20]:
        print(f"  FOD {fod} → CIP4 {fod_to_cip4[fod]}")
    
    # Check Computer Science specifically (FOD 2102)
    if 2102 in fod_to_cip4:
        print(f"\n✓ Computer Science: FOD 2102 → CIP4 {fod_to_cip4[2102]}")
    
    return fod_to_cip4


# =============================================================================
# STEP 3: LOAD AND PROCESS ACS PUMS DATA
# =============================================================================

def load_and_filter_acs(
    filepath: str,
    age_min: int = 22,
    age_max: int = 35
) -> pd.DataFrame:
    """
    Load and filter IPUMS ACS PUMS data.
    
    Your ACS columns: DEGFIELDD, OCCSOC, PERWT, AGE, EDUC, YEAR
    """
    print("\n" + "="*70)
    print("LOADING ACS PUMS DATA")
    print("="*70)
    
    acs = pd.read_csv(filepath)
    
    print(f"\nInitial sample: {len(acs):,} observations")
    
    # Filter
    acs_filtered = acs[
        (acs['AGE'] >= age_min) & 
        (acs['AGE'] <= age_max) &
        (acs['OCCSOC'].notna()) &
        (acs['DEGFIELDD'].notna())
    ].copy()
    
    print(f"Filtered sample: {len(acs_filtered):,} observations")
    print(f"  - Age {age_min}-{age_max}")
    print(f"  - Valid occupation (OCCSOC) and field of degree (DEGFIELDD)")
    
    # Clean SOC codes
    acs_filtered['soc_clean'] = acs_filtered['OCCSOC'].astype(str).str.replace('-', '').str.replace('.', '')
    
    print(f"\nUnique DEGFIELDD codes: {acs_filtered['DEGFIELDD'].nunique()}")
    print(f"Unique OCCSOC codes: {acs_filtered['OCCSOC'].nunique()}")
    
    return acs_filtered


# =============================================================================
# STEP 4: MAP FOD TO 4-DIGIT CIP AND MERGE WITH EXPOSURE
# =============================================================================

def process_acs_with_exposure(
    acs: pd.DataFrame,
    felten: pd.DataFrame,
    fod_to_cip4: Dict[int, str]
) -> pd.DataFrame:
    """
    Map ACS FOD codes to 4-digit CIP, merge with AI exposure scores.
    """
    print("\n" + "="*70)
    print("MAPPING FOD TO 4-DIGIT CIP AND MERGING AI EXPOSURE")
    print("="*70)
    
    # Map FOD to CIP4
    acs['CIP4'] = acs['DEGFIELDD'].map(fod_to_cip4)
    
    # Report mapping success
    n_mapped = acs['CIP4'].notna().sum()
    pct_mapped = 100 * n_mapped / len(acs)
    print(f"\nMapped {n_mapped:,}/{len(acs):,} observations to CIP4 ({pct_mapped:.1f}%)")
    
    # Check unmapped FODs
    if acs['CIP4'].isna().any():
        unmapped_fods = acs[acs['CIP4'].isna()]['DEGFIELDD'].value_counts().head(10)
        print("\nTop 10 unmapped FOD codes:")
        print(unmapped_fods)
    
    # Filter to successfully mapped
    acs_mapped = acs[acs['CIP4'].notna()].copy()
    
    # Merge with Felten AIOE scores on SOC code
    acs_with_exposure = acs_mapped.merge(
        felten[['soc_clean', 'AIOE']],
        on='soc_clean',
        how='left'
    )
    
    # Report merge success
    n_matched = acs_with_exposure['AIOE'].notna().sum()
    pct_matched = 100 * n_matched / len(acs_with_exposure)
    print(f"\nMatched {n_matched:,}/{len(acs_with_exposure):,} observations to AI exposure ({pct_matched:.1f}%)")
    
    # Handle missing AIOE scores (impute with mean)
    mean_aioe = acs_with_exposure['AIOE'].mean()
    n_missing = acs_with_exposure['AIOE'].isna().sum()
    if n_missing > 0:
        print(f"\n⚠ Imputing {n_missing} missing AIOE values with mean ({mean_aioe:.4f})")
        acs_with_exposure['AIOE'].fillna(mean_aioe, inplace=True)
    
    print(f"\nFinal sample: {len(acs_with_exposure):,} observations")
    print(f"Unique 4-digit CIP codes: {acs_with_exposure['CIP4'].nunique()}")
    
    return acs_with_exposure


# =============================================================================
# STEP 5: CALCULATE 4-DIGIT CIP-LEVEL AI EXPOSURE
# =============================================================================

def calculate_cip4_exposure(
    acs: pd.DataFrame,
    weight_var: str = 'PERWT'
) -> pd.DataFrame:
    """
    Calculate weighted average AI exposure by 4-digit CIP code.
    
    For each CIP4: AI_exposure = Σ [P(occupation|CIP4) × AIOE(occupation)]
    where P(occupation|CIP4) is weighted by person weights.
    """
    print("\n" + "="*70)
    print("CALCULATING 4-DIGIT CIP-LEVEL AI EXPOSURE SCORES")
    print("="*70)
    
    # Calculate weighted average by CIP4
    cip_exposure = acs.groupby('CIP4').apply(
        lambda x: pd.Series({
            'ai_exposure_score': np.average(x['AIOE'], weights=x[weight_var]),
            'n_obs': len(x),
            'n_weighted': x[weight_var].sum(),
            'min_exposure': x['AIOE'].min(),
            'max_exposure': x['AIOE'].max(),
            'std_exposure': np.sqrt(np.average((x['AIOE'] - np.average(x['AIOE'], weights=x[weight_var]))**2, 
                                                weights=x[weight_var]))
        })
    ).reset_index()
    
    print(f"\nCalculated AI exposure for {len(cip_exposure)} 4-digit CIP codes")
    print("\nAI Exposure Score Distribution:")
    print(cip_exposure['ai_exposure_score'].describe())
    
    # Show top and bottom CIPs
    print("\n\nTop 20 most AI-exposed majors (4-digit CIP):")
    top20 = cip_exposure.nlargest(20, 'ai_exposure_score')[['CIP4', 'ai_exposure_score', 'n_obs']]
    print(top20)
    
    print("\n\nBottom 20 least AI-exposed majors (4-digit CIP):")
    bottom20 = cip_exposure.nsmallest(20, 'ai_exposure_score')[['CIP4', 'ai_exposure_score', 'n_obs']]
    print(bottom20)
    
    return cip_exposure


# =============================================================================
# STEP 6: LOAD AND COMBINE ENROLLMENT DATA (2019-2025)
# =============================================================================

def load_and_combine_enrollment_data(
    filepath_2024: str, 
    filepath_2025: str
) -> pd.DataFrame:
    """
    Load and combine enrollment data from two sources with proper header handling.
    
    2024 file: Major Field (4-year, Undergrad) sheet, years 2019-2024
    2025 file: CIP Group Enrollment sheet, years 2020-2025 (filter to Undergraduate 4-year)
    
    Returns combined dataset with 4-digit CIP codes (2019-2025).
    """
    print("\n" + "="*70)
    print("LOADING AND COMBINING ENROLLMENT DATA (2019-2025)")
    print("="*70)
    
    # ===== LOAD 2024 FILE =====
    print("\nLoading 2019-2024 data from CTEESpring2024-Appendix.xlsx...")
    df_2024 = pd.read_excel(
        filepath_2024, 
        sheet_name='Major Field (4-year, Undergrad)',
        header=2  # Row 2 has the actual column headers
    )
    print(f"  Loaded {len(df_2024)} rows")
    
    # Rename columns for clarity
    df_2024 = df_2024.rename(columns={
        'Major Field Family (2-digit CIP)': 'CIP2',
        'Major Field Family (2-digit) Title': 'CIP2_title',
        'Major Field Group (4-digit CIP)': 'CIP4',
        'Major Field Group (4-digit) Title': 'CIP4_title'
    })
    
    # Get enrollment columns (years 2019-2024)
    years_2024 = [2019, 2020, 2021, 2022, 2023, 2024]
    enrollment_cols = [col for col in df_2024.columns if 'Enrollment' in str(col) and '% Change' not in str(col)]
    print(f"  Found {len(enrollment_cols)} enrollment columns for years 2019-2024")
    
    # Reshape to long format
    data_2024 = []
    for idx, row in df_2024.iterrows():
        cip4 = row['CIP4']
        if pd.isna(cip4) or cip4 == 'Total':
            continue
        for year, col in zip(years_2024, enrollment_cols):
            enrollment = row[col]
            if pd.notna(enrollment) and enrollment != '*':
                data_2024.append({
                    'CIP4': str(cip4).zfill(4) if str(cip4).isdigit() else cip4,
                    'year': year,
                    'enrollment': float(enrollment)
                })
    
    df_2024_long = pd.DataFrame(data_2024)
    print(f"  Reshaped to {len(df_2024_long)} observations")
    
    # ===== LOAD 2025 FILE =====
    print("\nLoading 2020-2025 data from CTEESpring2025-DataAppendix.xlsx...")
    df_2025 = pd.read_excel(
        filepath_2025,
        sheet_name='CIP Group Enrollment',
        header=2  # Row 2 has the actual column headers
    )
    print(f"  Loaded {len(df_2025)} rows")
    
    # Filter to Undergraduate 4-year only
    df_2025 = df_2025[df_2025['Award Level and Institution Type'] == 'Undergraduate 4-year'].copy()
    print(f"  Filtered to {len(df_2025)} Undergraduate 4-year rows")
    
    # Rename columns
    df_2025 = df_2025.rename(columns={
        'Major Field Family \n(2-digit CIP)': 'CIP2',
        'Major Field Family \n(2-digit CIP) Title': 'CIP2_title',
        'Major Field Group \n(4-digit CIP)': 'CIP4',
        'Major Field Group \n(4-digit CIP) Title': 'CIP4_title'
    })
    
    # Get enrollment columns (years 2020-2025)
    # The enrollment columns alternate: Enrollment, % Change, Enrollment, % Change...
    # Columns 5, 6, 8, 10, 12, 14 correspond to years 2020-2025
    years_2025 = [2020, 2021, 2022, 2023, 2024, 2025]
    enrollment_col_indices = [5, 6, 8, 10, 12, 14]
    
    # Reshape to long format
    data_2025 = []
    for idx, row in df_2025.iterrows():
        cip4 = row['CIP4']
        if pd.isna(cip4) or cip4 == 'Total':
            continue
        for year, col_idx in zip(years_2025, enrollment_col_indices):
            enrollment = row.iloc[col_idx]
            if pd.notna(enrollment) and enrollment != '*':
                data_2025.append({
                    'CIP4': str(cip4).zfill(4) if str(cip4).isdigit() else cip4,
                    'year': year,
                    'enrollment': float(enrollment)
                })
    
    df_2025_long = pd.DataFrame(data_2025)
    print(f"  Reshaped to {len(df_2025_long)} observations")
    
    # ===== COMBINE DATASETS =====
    print("\nCombining datasets...")
    
    # For overlapping years (2020-2024), use 2025 file data (more recent)
    df_2024_unique = df_2024_long[df_2024_long['year'] == 2019].copy()
    
    enrollment = pd.concat([df_2024_unique, df_2025_long], axis=0, ignore_index=True)
    enrollment = enrollment.sort_values(['CIP4', 'year']).reset_index(drop=True)
    
    print(f"\n✓ Combined dataset: {len(enrollment)} observations")
    print(f"  Years: {sorted(enrollment['year'].unique())}")
    print(f"  Unique 4-digit CIP codes: {enrollment['CIP4'].nunique()}")
    
    # Summary stats
    print("\nTotal enrollment by year:")
    yearly_enrollment = enrollment.groupby('year')['enrollment'].sum()
    for year, total in yearly_enrollment.items():
        print(f"  {year}: {total:,.0f}")
    
    return enrollment


# =============================================================================
# STEP 7: MERGE AND FINALIZE
# =============================================================================

def merge_enrollment_exposure(
    enrollment: pd.DataFrame,
    cip_exposure: pd.DataFrame
) -> pd.DataFrame:
    """
    Merge enrollment data with AI exposure scores.
    """
    print("\n" + "="*70)
    print("MERGING ENROLLMENT WITH AI EXPOSURE")
    print("="*70)
    
    # Merge on 4-digit CIP code
    df_final = enrollment.merge(
        cip_exposure[['CIP4', 'ai_exposure_score', 'n_obs']],
        on='CIP4',
        how='left'
    )
    
    # Report merge success
    n_matched = df_final['ai_exposure_score'].notna().sum()
    pct_matched = 100 * n_matched / len(df_final)
    print(f"\nMatched {n_matched}/{len(df_final)} enrollment records ({pct_matched:.1f}%)")
    
    # Create treatment variables
    median_exposure = df_final['ai_exposure_score'].median()
    df_final['high_ai_exposure'] = (
        df_final['ai_exposure_score'] > median_exposure
    ).astype(int)
    
    # Standardized exposure
    df_final['ai_exposure_std'] = (
        (df_final['ai_exposure_score'] - df_final['ai_exposure_score'].mean()) /
        df_final['ai_exposure_score'].std()
    )
    
    # Terciles
    df_final['ai_exposure_tercile'] = pd.qcut(
        df_final['ai_exposure_score'],
        q=3,
        labels=['Low', 'Medium', 'High'],
        duplicates='drop'
    )
    
    # Create log enrollment
    df_final['log_enrollment'] = np.log(df_final['enrollment'] + 1)
    
    print("\n\nFinal dataset:")
    print(df_final.head(20))
    print(f"\nShape: {df_final.shape}")
    
    return df_final


# =============================================================================
# STEP 8: VISUALIZATION
# =============================================================================

def create_descriptive_plots(df: pd.DataFrame, output_path: str = 'enrollment_trends_4digit.png'):
    """
    Create descriptive visualizations for 4-digit CIP analysis.
    """
    print("\n" + "="*70)
    print("CREATING VISUALIZATIONS")
    print("="*70)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Enrollment trends by AI exposure group
    ax1 = axes[0, 0]
    trend_data = df.groupby(['year', 'high_ai_exposure'])['enrollment'].sum().reset_index()
    for group in [0, 1]:
        data = trend_data[trend_data['high_ai_exposure'] == group]
        label = 'High AI Exposure' if group else 'Low AI Exposure'
        ax1.plot(data['year'], data['enrollment'], marker='o', label=label, linewidth=2)
    ax1.set_xlabel('Year', fontsize=12)
    ax1.set_ylabel('Total Enrollment', fontsize=12)
    ax1.set_title('Enrollment Trends by AI Exposure (4-digit CIP)', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)
    ax1.axvline(2022.5, color='red', linestyle='--', alpha=0.5, label='ChatGPT Launch')
    
    # 2. Distribution of AI exposure
    ax2 = axes[0, 1]
    cip_scores = df.groupby('CIP4')['ai_exposure_score'].first()
    ax2.hist(cip_scores, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    ax2.axvline(cip_scores.median(), color='red', linestyle='--', linewidth=2, label='Median')
    ax2.set_xlabel('AI Exposure Score', fontsize=12)
    ax2.set_ylabel('Number of 4-digit CIP Codes', fontsize=12)
    ax2.set_title('Distribution of AI Exposure Across Majors', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # 3. Scatter: enrollment growth vs exposure
    ax3 = axes[1, 0]
    first_year = df['year'].min()
    last_year = df['year'].max()
    
    growth_data = []
    for cip in df['CIP4'].unique():
        cip_data = df[df['CIP4'] == cip]
        enroll_first = cip_data[cip_data['year'] == first_year]['enrollment'].values
        enroll_last = cip_data[cip_data['year'] == last_year]['enrollment'].values
        if len(enroll_first) > 0 and len(enroll_last) > 0 and enroll_first[0] > 0:
            growth = (enroll_last[0] - enroll_first[0]) / enroll_first[0] * 100
            exposure = cip_data['ai_exposure_score'].iloc[0] if len(cip_data) > 0 else None
            if exposure is not None and pd.notna(exposure):
                growth_data.append({'CIP4': cip, 'growth_rate': growth, 'ai_exposure': exposure})
    
    growth_df = pd.DataFrame(growth_data)
    if len(growth_df) > 0:
        ax3.scatter(growth_df['ai_exposure'], growth_df['growth_rate'], alpha=0.6, s=30)
        ax3.set_xlabel('AI Exposure Score', fontsize=12)
        ax3.set_ylabel(f'Enrollment Growth Rate ({first_year}-{last_year}, %)', fontsize=12)
        ax3.set_title('Growth Rate vs AI Exposure', fontsize=14, fontweight='bold')
        ax3.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
        ax3.grid(alpha=0.3)
        
        # Add correlation
        corr = growth_df[['ai_exposure', 'growth_rate']].corr().iloc[0, 1]
        ax3.text(0.05, 0.95, f'Correlation: {corr:.3f}', 
                transform=ax3.transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # 4. Enrollment by tercile over time
    ax4 = axes[1, 1]
    tercile_data = df.groupby(['year', 'ai_exposure_tercile'])['enrollment'].sum().reset_index()
    for tercile in ['Low', 'Medium', 'High']:
        data = tercile_data[tercile_data['ai_exposure_tercile'] == tercile]
        if len(data) > 0:
            ax4.plot(data['year'], data['enrollment'], marker='o', label=f'{tercile} Exposure', linewidth=2)
    ax4.set_xlabel('Year', fontsize=12)
    ax4.set_ylabel('Total Enrollment', fontsize=12)
    ax4.set_title('Enrollment by AI Exposure Tercile', fontsize=14, fontweight='bold')
    ax4.legend()
    ax4.grid(alpha=0.3)
    ax4.axvline(2022.5, color='red', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"\n✓ Saved plots to {output_path}")
    plt.close()


# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """
    Main execution function.
    """
    
    print("\n" + "#"*70)
    print("# AI EXPOSURE AND ENROLLMENT ANALYSIS - 4-DIGIT CIP")
    print("#"*70 + "\n")
    
    # FILE PATHS
    FELTEN_PATH = '/Users/jeffreyohl/Dropbox/CollegeMajorData/FeltenEtAl/AIOE_DataAppendix.xlsx'
    CROSSWALK_PATH = '/Users/jeffreyohl/Dropbox/CollegeMajorData/Crosswalks/crosswalk_handout.xlsx'
    ACS_PATH = '/Users/jeffreyohl/Dropbox/CollegeMajorData/IPUMS/usa_00008.csv'
    ENROLLMENT_PATH_2025 = '/Users/jeffreyohl/Dropbox/CollegeMajorData/National Student Clearinghouse Data/CTEESpring2025-DataAppendix.xlsx'
    ENROLLMENT_PATH_2024 = '/Users/jeffreyohl/Dropbox/CollegeMajorData/National Student Clearinghouse Data/CTEESpring2024-Appendix.xlsx'
    OUTPUT_DIR = '/Users/jeffreyohl/Dropbox/CollegeMajorData/output'
    
    try:
        # Step 1: Load Felten data
        felten = load_felten_data(FELTEN_PATH)
        
        # Step 2: Load FOD to 4-digit CIP crosswalk
        fod_to_cip4 = load_fod_cip4_crosswalk(CROSSWALK_PATH)
        
        # Step 3-5: Process ACS and calculate exposure scores
        print("\n" + "="*70)
        print("PROCESSING ACS DATA")
        print("="*70)
        acs = load_and_filter_acs(ACS_PATH)
        acs_with_exposure = process_acs_with_exposure(acs, felten, fod_to_cip4)
        cip_exposure = calculate_cip4_exposure(acs_with_exposure)
        
        # Save exposure scores
        cip_exposure.to_csv(f'{OUTPUT_DIR}/cip4_ai_exposure_scores.csv', index=False)
        print(f"\n✓ Saved exposure scores to {OUTPUT_DIR}/cip4_ai_exposure_scores.csv")
        
        # Step 6: Load and combine enrollment data (2019-2025)
        enrollment = load_and_combine_enrollment_data(ENROLLMENT_PATH_2024, ENROLLMENT_PATH_2025)
        
        # Step 7: Merge enrollment with exposure
        df_final = merge_enrollment_exposure(enrollment, cip_exposure)
        
        # Save final dataset
        df_final.to_csv(f'{OUTPUT_DIR}/enrollment_with_ai_exposure_4digit.csv', index=False)
        print(f"\n✓ Saved final dataset to {OUTPUT_DIR}/enrollment_with_ai_exposure_4digit.csv")
        
        # Step 8: Create visualizations
        create_descriptive_plots(df_final, f'{OUTPUT_DIR}/enrollment_trends_4digit.png')
        
        print("\n" + "#"*70)
        print("# DATA PREPARATION COMPLETE (4-DIGIT CIP)")
        print("#"*70)
        print("\nNext steps:")
        print("1. Review cip4_ai_exposure_scores.csv to validate exposure scores")
        print("2. Check enrollment_with_ai_exposure_4digit.csv for data quality")
        print("3. Run econometric_analysis.py for DiD and event study")
        print("\n4-digit CIP analysis provides:")
        print("  - Computer Science (1107) vs Information Systems (1104)")
        print("  - Business Administration (5202) vs Finance (5208) vs Accounting (5203)")  
        print("  - More granular treatment effects and heterogeneity analysis")
        
    except FileNotFoundError as e:
        print(f"\n❌ Error: File not found - {e}")
        print("\nPlease check that all data files exist at the specified paths:")
        print(f"  - Felten: {FELTEN_PATH}")
        print(f"  - Crosswalk: {CROSSWALK_PATH}")
        print(f"  - ACS: {ACS_PATH}")
        print(f"  - Enrollment 2024: {ENROLLMENT_PATH_2024}")
        print(f"  - Enrollment 2025: {ENROLLMENT_PATH_2025}")
    except Exception as e:
        print(f"\n❌ Error occurred: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


######################################################################
# AI EXPOSURE AND ENROLLMENT ANALYSIS - 4-DIGIT CIP
######################################################################

LOADING FELTEN AIOE DATA

Loaded 774 occupations
AIOE range: -2.67 to 1.53

LOADING FOD TO 4-DIGIT CIP CROSSWALK

Mapped 191 FOD codes to 4-digit CIP codes

Sample mappings:
  FOD 1100 → CIP4 0100
  FOD 1101 → CIP4 0106
  FOD 1102 → CIP4 0100
  FOD 1103 → CIP4 0109
  FOD 1104 → CIP4 0110
  FOD 1105 → CIP4 0111
  FOD 1106 → CIP4 0112
  FOD 1199 → CIP4 0108
  FOD 1301 → CIP4 0301
  FOD 1302 → CIP4 0305
  FOD 1303 → CIP4 0302
  FOD 1401 → CIP4 0400
  FOD 1501 → CIP4 0501
  FOD 1901 → CIP4 0901
  FOD 1902 → CIP4 0904
  FOD 1903 → CIP4 0907
  FOD 1904 → CIP4 0909
  FOD 2001 → CIP4 1003
  FOD 2100 → CIP4 1101
  FOD 2101 → CIP4 1102

✓ Computer Science: FOD 2102 → CIP4 1100

PROCESSING ACS DATA

LOADING ACS PUMS DATA


In [ ]:
# Run main analysis
main()